In [ ]:
from depmapomics.config import *
from depmapomics import tracker
from depmapomics import upload
from depmapomics import mutations
from genepy.utils import helper as h

In [ ]:
wgs_mat = mutations.generateGermlineMatrix(refworkspace=WGSWORKSPACE,
        vcfdir=WGSVCFDIR,
        savedir="output/" + SAMPLESETNAME + "/",
        filename="binary_mutguides_wgs.tsv.gz",
        bed_location=GUIDESBED,)

In [ ]:
### test running bcftools w/ gcs data vs. w/ local data
vcfslist = ["gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/4b0d5819-84a3-4a55-a23b-cd169cc9caf9/Mutect2/d05388cc-7977-4191-80ac-8656a7d451f0/call-Filter/CDS-00Nrci.wgs_ccle-filtered.vcf.gz",
           "gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/4b0d5819-84a3-4a55-a23b-cd169cc9caf9/Mutect2/2b5664d2-fa3d-4bf9-b60a-5cfa81209db3/call-Filter/CDS-0b4jFH.wgs_ccle-filtered.vcf.gz",
           "gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/8170451d-2504-473b-9087-0088d47ead7b/Mutect2/504c5aa7-90dc-491f-8386-12239b3ab7b5/call-Filter/CDS-0bV15m.wgs_ccle-filtered.vcf.gz"]

In [ ]:
# 1. directly pulling from gs bucket
bed_location=GUIDESBED
vcfdir=WGSVCFDIR
cmd = [
        "export GCS_OAUTH_TOKEN=`gcloud auth application-default print-access-token`"
        + " && bcftools query \
  --exclude \"FILTER!='PASS'&GT!='mis'&GT!~'\.'\" \
  --regions-file "
        + bed_location
        + " \
  --format '%CHROM\\t%POS\\t%END\\t%ALT{0}\n' "
        + sam
        + " >\
 "
        + vcfdir
        + "loc_"
        + sam.split("/")[-1].split(".")[0]
        + ".bed"
        for sam in vcfslist
    ]

In [ ]:
cmd

In [ ]:
from datetime import datetime

start=datetime.now()

h.parrun(cmd, cores=16)

print(datetime.now()-start)

In [ ]:
# 2. downloading vcfs and indices
vcfdir = "/tmp/vcfs_wgs_test/"
cmd = [
        "gsutil cp "
        + sam
        + " "
        + vcfdir
        + sam.split("/")[-1]
        + " && gsutil cp "
        + sam + ".tbi"
        + " "
        + vcfdir
        + sam.split("/")[-1] + ".tbi"
        + " && bcftools query \
  --exclude \"FILTER!='PASS'&GT!='mis'&GT!~'\.'\" \
  --regions-file "
        + bed_location
        + " \
  --format '%CHROM\\t%POS\\t%END\\t%ALT{0}\n' "
        + vcfdir
        + sam.split("/")[-1]
        + " >\
 "
        + vcfdir
        + "loc_"
        + sam.split("/")[-1].split(".")[0]
        + ".bed &&\
 rm "
        + vcfdir
        + sam.split("/")[-1]
        + "*"
        for sam in vcfslist
    ]

In [ ]:
cmd

In [ ]:
from datetime import datetime

start=datetime.now()

h.parrun(cmd, cores=16)

print(datetime.now()-start)